In [1]:
import logging
import random
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait as wait


from selenium.webdriver.common.action_chains import ActionChains

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from typing import Dict
from orb.utils import GetUserAgent
from orb.utils import GetProxies
from webdriver_manager.chrome import ChromeDriverManager

import logging
import random

from selenium.webdriver.common.by import By

from orb.spinner.utils.spoofing import DriverSpoofing

log = logging.getLogger(__name__)


class OrbDriver:

    """
    This class builds an instance of a chrome driver utilising a random proxy and headers (which can be rotated).
    The driver instance can be created using the get_webdriver method.
    """

    def __init__(self, *args, **kwargs):
        self.driver_spoof = DriverSpoofing()

        if kwargs.get('headless'):
            self.headless = kwargs['headless']
        else:
            self.headless = None

        self.driver_install = None

    @property
    def random_user_agent(self) -> 'GetUserAgent':
        return GetUserAgent().headers_dict['User-Agent']

    @property
    def random_proxy(self) -> 'GetProxies':
        return GetProxies().proxy_dict['https']

    def _webdriver_options_init(self):

        # Initialise webdriver options
        self.webdriver_options = Options()

        # Default webdriver options
        # Disable javascript
        self.webdriver_options.add_argument("--disable-javascript")

        # Set to headless if applicable
        if self.headless:
            self.webdriver_options.add_argument("--headless")

        # Set the user agent string
        user_agent = self.random_user_agent
        log.info(f"Initialising webdriver with user-agent: {user_agent} ")
        self.webdriver_options.add_argument(f"user-agent={user_agent}")

        # Set the proxy string
        proxy = self.random_proxy
        log.info(f"Initialising webdriver with proxy: {proxy} ")
        self.webdriver_options.add_argument(f'--proxy-server={proxy}')

    def change_driver_viewport(self):

        self.driver_spoof.change_viewport_size(
            driver=self.driver
        )

    def driver_init__(self):
        # Always install the latest web driver
        self.driver_install = ChromeDriverManager().install()

    def get_webdriver(self) -> webdriver.Chrome:
        
        # Initialise webdriver options
        self._webdriver_options_init()

        # Always install the latest web driver
        # Initialize the webdriver with the options
        if not self.driver_install:
            self.driver_init__()
            self.driver = webdriver.Chrome(self.driver_install, options=self.webdriver_options)

        self.driver = webdriver.Chrome(options=self.webdriver_options)
        
        return self.driver

    def get_url(self, driver: webdriver.Chrome = None):
        if not driver:
            self.driver = self.get_webdriver()
        else:
            self.driver = driver

        

In [3]:
orb = OrbDriver()

In [5]:
driver = orb.get_webdriver()

/tmp/ipykernel_17657/2414776706.py:86: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(self.driver_install, options=self.webdriver_options)


In [6]:
driver.get('https://the-internet.herokuapp.com/')

In [3]:
import concurrent.futures
import functools
import time
from typing import Callable

import logging
from IPython.display import display, HTML

# Create a logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Create a log handler that outputs log messages to the notebook output
log_handler = logging.StreamHandler()
log_handler.setLevel(logging.DEBUG)
log_format = logging.Formatter('%(levelname)s - %(message)s')
log_handler.setFormatter(log_format)

# Add the log handler to the logger
logger.addHandler(log_handler)


class OrbFailSafe:

    MAX_ATTEMPTS = 3
    TIMEOUT: int = 20

    def __init__(self, driver: webdriver.Chrome = None) -> None:
        if not driver:
            self.driver = None
            self.__init__driver()

    def __init__driver(self):

        if self.driver:
            self.driver.close()

        orb = OrbDriver()
        self.driver = orb.get_webdriver()

    def driver_timeout(self, func: Callable[[str], None], **kwargs):

        # Creating a thread pool executor
        executor = concurrent.futures.ThreadPoolExecutor()

        # Creating a partial function with the variable
        partial_function = functools.partial(func, **kwargs)

        # Submitting the partial function to the executor
        future = executor.submit(partial_function)

        # Waiting for the function to complete or timeout
        try:
            future.result(timeout=self.TIMEOUT)
            log.info(f"{func.__name__}Function completed. Result:")
        except concurrent.futures.TimeoutError:
            log.error(f"{func.__name__} Function timed out. Cancelling...")
            future.cancel()

            # If fails, define new instance and try again
            self.__init__driver()
            return

        except Exception as e:
            log.debug("An error occurred:", e)
            return

        # Shutting down the executor
        executor.shutdown()
        return True

    def _get_driver(self, url: str):
        self.driver.get(url=url)

    def get_url(self, url: str):

        for _ in range(0, self.MAX_ATTEMPTS):
            if self.driver_timeout(func=self._get_driver, url=url):
                break
            else:
                self.driver.close()
                self.__init__driver()




In [4]:
orb = OrbFailSafe()

INFO - Initialising webdriver with user-agent: Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/46.0.2486.0 Safari/537.36 Edge/13.9200 
DEBUG - Starting new HTTPS connection (1): free-proxy-list.net:443
DEBUG - https://free-proxy-list.net:443 "GET / HTTP/1.1" 200 None
DEBUG - Starting new HTTP connection (1): 3.36.130.175:80
DEBUG - http://3.36.130.175:80 "GET http://www.example.com/ HTTP/1.1" 200 1227
INFO - 3.36.130.175:80 Proxy is working!
INFO - Initialising webdriver with proxy: 3.36.130.175:80 
INFO - ====== WebDriver manager ======
INFO - Driver [/home/tom/.wdm/drivers/chromedriver/linux64/110.0.5481.77/chromedriver] found in cache
/tmp/ipykernel_15318/122579604.py:86: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(self.driver_install, options=self.webdriver_options)
DEBUG - Started executable: `/home/tom/.wdm/drivers/chromedriver/linux64/110.0.5481.77/chromedriver` in a ch

In [6]:
orb.get_url(url='https://the-internet.herokuapp.com/')

DEBUG - POST http://localhost:54407/session/5eb659fc0e54d87133f16b60e5e099f2/url {"url": "https://the-internet.herokuapp.com/"}
DEBUG - http://localhost:54407 "POST /session/5eb659fc0e54d87133f16b60e5e099f2/url HTTP/1.1" 200 14
DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
DEBUG - Finished Request
INFO - _get_driverFunction completed. Result:


In [19]:
orb = OrbDriver()
driver = orb.get_webdriver()

INFO - Initialising webdriver with user-agent: Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_3; de-de) AppleWebKit/531.22.7 (KHTML, like Gecko) Version/4.0.5 Safari/531.22.7 
INFO - Initialising webdriver with user-agent: Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_3; de-de) AppleWebKit/531.22.7 (KHTML, like Gecko) Version/4.0.5 Safari/531.22.7 
DEBUG - Starting new HTTPS connection (1): free-proxy-list.net:443
DEBUG - Starting new HTTPS connection (1): free-proxy-list.net:443
DEBUG - https://free-proxy-list.net:443 "GET / HTTP/1.1" 200 None
DEBUG - https://free-proxy-list.net:443 "GET / HTTP/1.1" 200 None
INFO - Initialising webdriver with proxy: 86.98.0.231:8080 
INFO - Initialising webdriver with proxy: 86.98.0.231:8080 
INFO - ====== WebDriver manager ======
INFO - ====== WebDriver manager ======
INFO - Driver [/home/tom/.wdm/drivers/chromedriver/linux64/110.0.5481.77/chromedriver] found in cache
INFO - Driver [/home/tom/.wdm/drivers/chromedriver/linux64/110.0.5481.77/chromedri

In [20]:
orb.random_proxy

DEBUG - Starting new HTTPS connection (1): free-proxy-list.net:443
DEBUG - Starting new HTTPS connection (1): free-proxy-list.net:443
DEBUG - https://free-proxy-list.net:443 "GET / HTTP/1.1" 200 None
DEBUG - https://free-proxy-list.net:443 "GET / HTTP/1.1" 200 None


'8.219.97.248:80'

In [21]:
GetProxies().proxy_dict['https']

DEBUG - Starting new HTTPS connection (1): free-proxy-list.net:443
DEBUG - Starting new HTTPS connection (1): free-proxy-list.net:443
DEBUG - https://free-proxy-list.net:443 "GET / HTTP/1.1" 200 None
DEBUG - https://free-proxy-list.net:443 "GET / HTTP/1.1" 200 None


'64.225.8.121:9985'

In [38]:
import requests

def test_proxy(proxies):
    try:
        # Define the URL you want to access through the proxy
        url = 'http://www.example.com'

        # Make a request using the proxy
        response = requests.get(url, proxies=proxies, timeout=5)

        # Check the response status code
        if response.status_code == 200:
            print("Proxy is working!")
        else:
            print("Proxy is not working.")
    except requests.exceptions.RequestException:
        print("Unable to connect to the proxy.")



In [44]:
# Usage example
test_proxy(proxies)

DEBUG - Starting new HTTP connection (1): 202.86.138.18:8080
DEBUG - Starting new HTTP connection (1): 202.86.138.18:8080
DEBUG - http://202.86.138.18:8080 "GET http://www.example.com/ HTTP/1.1" 200 648
DEBUG - http://202.86.138.18:8080 "GET http://www.example.com/ HTTP/1.1" 200 648


Proxy is working!


In [39]:
proxies = GetProxies().proxy_dict
proxies

DEBUG - Starting new HTTPS connection (1): free-proxy-list.net:443
DEBUG - Starting new HTTPS connection (1): free-proxy-list.net:443
DEBUG - https://free-proxy-list.net:443 "GET / HTTP/1.1" 200 None
DEBUG - https://free-proxy-list.net:443 "GET / HTTP/1.1" 200 None


{'http': '202.86.138.18:8080', 'https': '202.86.138.18:8080'}

In [36]:
proxies

{'http': '202.76.60.166:9443', 'https': '202.76.60.166:9443'}

In [23]:
from typing import Callable, Tuple, Union
import copy
import selenium

def assembeler(func: Callable) -> Callable:

    def _copy(self, *args, **kwargs):
        self_copy = copy.copy(self) if getattr(self, "immutable", True) else self
        result = func(self_copy, *args, **kwargs)
        if not result:
            return self_copy
        return result
    return _copy


class OrbBuilder:

    LOCATOR_DICT = {
        'id': By.ID,
        'xpath': By.XPATH,
        'link_text': By.LINK_TEXT,
        'partial_link_text': By.PARTIAL_LINK_TEXT,
        'name': By.NAME,
        'tag_name': By.TAG_NAME,
        'class_name': By.CLASS_NAME,
        'css_selector': By.CSS_SELECTOR
    }

    # Intervals for which the driver will randomly wait for a time interval between
    WAITING_INTERVALS: Tuple[Union[int, float], Union[int, float]] = (0.1, 3)

    def __init__(self, driver: webdriver.Chrome) -> None:
        self.driver = driver
        self.instructions = []

    def wait(self) -> int:
        return random.uniform(
            self.WAITING_INTERVALS[0], self.WAITING_INTERVALS[0])

    @assembeler
    def get_url(self, url: str):
        self.driver.get(url)

    @assembeler
    def find_elements_with_driver(self, locator: str, value: str):

        if not self.LOCATOR_DICT.get(locator):
            log.error(f"Locator strategy must be one of the following: {', '.join(self.LOCATOR_DICT.keys())}")

        self.elements = WebDriverWait(
            self.driver, self.wait()).until(
                ec.visibility_of_element_located((self.LOCATOR_DICT[locator], value)))

    @assembeler
    def find_elements_with_selenium_obj(self, obj: selenium.webdriver.remote.webelement.WebElement, locator: str, value: str):

        if not self.LOCATOR_DICT.get(locator):
            log.error(f"Locator strategy must be one of the following: {', '.join(self.LOCATOR_DICT.keys())}")

        self.elements = WebDriverWait(
            self.driver, self.wait()).until(
                ec.visibility_of_element_located((self.LOCATOR_DICT[locator], value)))   

In [25]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="71e00a9bcc359d242d56b236c186c4d6")>

In [ ]:
type(c.elements)

selenium.webdriver.remote.webelement.WebElement

In [20]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="71e00a9bcc359d242d56b236c186c4d6")>

In [27]:
c.instructions

[]

In [46]:
driver